# BERT

In [ ]:
!pip install sentence-transformers

In [23]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn. preprocessing import StandardScaler

In [3]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [11]:
df =pd.read_csv("https://raw.githubusercontent.com/KrzysztofSzerenos/DMML2022_-Hublot/main/data/training_data.csv",)

In [12]:
sentences = df['sentence'].tolist()

In [13]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [7]:
model =  SentenceTransformer("dangvantuan/sentence-camembert-large")

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/683 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/809k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/298 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/400 [00:00<?, ?B/s]

In [14]:
query = "I love Data Mining !"
query_vec = model.encode([query])[0]

In [39]:
for sent in sentences:
  sim = cosine(query_vec, model.encode([sent])[0])
  print("Sentence = ", sent, "; similarity = ", sim)

Sentence =  Les coûts kilométriques réels peuvent diverger sensiblement des valeurs moyennes en fonction du moyen de transport utilisé, du taux d'occupation ou du taux de remplissage, de l'infrastructure utilisée, de la topographie des lignes, du flux de trafic, etc. ; similarity =  0.103812486
Sentence =  Le bleu, c'est ma couleur préférée mais je n'aime pas le vert! ; similarity =  0.13428228
Sentence =  Le test de niveau en français est sur le site Internet de l'école. ; similarity =  0.03351088
Sentence =  Est-ce que ton mari est aussi de Boston? ; similarity =  0.037612684


KeyboardInterrupt: ignored

In [19]:
df_bert =pd.read_csv("https://raw.githubusercontent.com/KrzysztofSzerenos/DMML2022_-Hublot/main/data/bert2.csv",)
df_bert.shape

(4800, 1)

In [20]:
df['bert'] = df_bert['bert']
df.tail(10)

,id,sentence,difficulty,bert
4790,4790,Merci pour cette histoire et bonne année !,A1,0.173168
4791,4791,La marche était cassée et je suis tombé par te...,B1,0.079947
4792,4792,Les villages situés à mi-côte sont cachés par ...,C1,-0.006260
4793,4793,La réduction du dioxyde de carbone par l'eau n...,B2,-0.000044
4794,4794,Elle connaissait à présent la petitesse des pa...,C1,0.013849
4795,4795,"C'est pourquoi, il décida de remplacer les hab...",B2,-0.028824
4796,4796,Il avait une de ces pâleurs splendides qui don...,C1,0.044852
4797,4797,"Et le premier samedi de chaque mois, venez ren...",A2,0.113623
4798,4798,Les coûts liés à la journalisation n'étant pas...,C2,-0.042015
4799,4799,"Sur le sable, la mer haletait de toute la resp...",C2,-0.049908


In [21]:
df['word_count'] = df['sentence'].apply(lambda n: len(n.split()))
df['characters_count'] = df['sentence'].apply(lambda x: sum(len(word) for word in str(x).split(" ")))
df.head()

,id,sentence,difficulty,bert,word_count,characters_count
0,0,Les coûts kilométriques réels peuvent diverger...,C1,0.001702,38,218
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1,0.215746,12,51
2,2,Le test de niveau en français est sur le site ...,A1,-0.019108,13,54
3,3,Est-ce que ton mari est aussi de Boston?,A1,0.051101,8,33
4,4,"Dans les écoles de commerce, dans les couloirs...",B1,-0.006344,34,176


# Encoder

In [24]:
oe = OrdinalEncoder()
oe.set_params(categories=[['A1', 'A2', 'B1','B2','C1','C2']])
oe_customer_class = oe.fit_transform(df[['difficulty']])
oe.categories_

[array(['A1', 'A2', 'B1', 'B2', 'C1', 'C2'], dtype=object)]

In [25]:
oe_customer_class = pd.DataFrame(oe_customer_class, columns=['oe_difficulty']).astype('int')

In [26]:
df = pd.concat([df, oe_customer_class], axis=1) 
df

,id,sentence,difficulty,bert,word_count,characters_count,oe_difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,C1,0.001702,38,218,4
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1,0.215746,12,51,0
2,2,Le test de niveau en français est sur le site ...,A1,-0.019108,13,54,0
3,3,Est-ce que ton mari est aussi de Boston?,A1,0.051101,8,33,0
4,4,"Dans les écoles de commerce, dans les couloirs...",B1,-0.006344,34,176,2
...,...,...,...,...,...,...,...
4795,4795,"C'est pourquoi, il décida de remplacer les hab...",B2,-0.028824,26,140,3
4796,4796,Il avait une de ces pâleurs splendides qui don...,C1,0.044852,21,98,4
4797,4797,"Et le premier samedi de chaque mois, venez ren...",A2,0.113623,14,67,1
4798,4798,Les coûts liés à la journalisation n'étant pas...,C2,-0.042015,32,195,5


# vectorization

Linear regression model

In [32]:
X = df[['bert','word_count','characters_count']]
y = df['oe_difficulty']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=0)
np.unique(y, return_counts=True)

(array([0, 1, 2, 3, 4, 5]), array([813, 795, 795, 792, 798, 807]))

In [33]:
df.reset_index(drop=True, inplace=True)

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()  # vectorization
vectorizer.fit(X,y)
X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [38]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
classifier = LogisticRegressionCV()
classifier.fit(X_train_vec, y_train)
log_reg_pred = classifier.predict(X_kag_set)

ValueError: ignored

In [ ]:
submission_dataframe = pd.DataFrame(log_reg_pred, columns=['Difficulty'])
print(submission_dataframe)
submission_dataframe.to_csv('HublotKAGGLECV.csv')